<a href="https://colab.research.google.com/github/faranak5171/Deep-Learning-projects/blob/master/Dogs_vs_Cats_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
from torchvision import transforms, datasets, models
from torch import nn, optim
import torch.nn.functional as F

import warnings

In [3]:
%%capture
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! cp kaggle.json ~/.kaggle/
! kaggle competitions download dogs-vs-cats
! unzip dogs-vs-cats
! unzip train.zip -d train
! unzip test1.zip -d test

In [25]:
train_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomRotation(30),
                                      transforms.RandomHorizontalFlip()])

test_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                      transforms.CenterCrop(224),
                                      transforms.Resize(255)])

In [26]:
trainset = datasets.ImageFolder('train', transform=train_transform)
testset = datasets.ImageFolder('test', transform=train_transform)

In [29]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32)
testLoader = torch.utils.data.DataLoader(testset, batch_size=32)

In [30]:
model = models.densenet121(pretrained=True)

for param in model.parameters():
  param.requires_grad = False

model.classifier = nn.Sequential(nn.Linear(1024,256),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(256,2),
                           nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

In [31]:
%%capture
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
epochs = 1

for epoch in range(epochs):
  for images, labels in trainloader:
    